In [1]:
import importlib
import xarray as xr
import pandas as pd
import numpy as np
import sys

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import jlat_utils as jlat
from ecpaper_utils import calendar_utils as cal

importlib.reload(read)
importlib.reload(jlat)
importlib.reload(cal)

import warnings
warnings.filterwarnings("ignore")

Set up model names, dates and CLIVAR_LE location

In [2]:
modnames=['CanESM2','CSIRO-Mk3-6-0','CESM1-CAM5','GFDL-CM3','MPI-ESM']
modnames_small=['canesm2','csiro_mk36','cesm','gfdl_cm3','mpi']
basedir="/glade/collections/cdg/data/CLIVAR_LE/"
ybegp=1979 ; monbegp=1 ; yendp=2014 ; monendp=12
ybegf=2070 ; monbegf=1 ; yendf=2099 ; monendf=12
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

nmonthsp=(yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf=(yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

plevuse="70000"

Loop over models and members and output JJA climatology ofzonal mean zonal wind at 700hPa 

In [3]:
for imod in range(len(modnames)):
    print(modnames[imod])
    moddir=basedir+modnames_small[imod]+"_lens/Amon/ua/"
    memfiles = !cd $moddir && ls *rcp85*.nc
    nmems=len(memfiles)
    
    for imem in range(len(memfiles)):
        fname=memfiles[imem]
        print(fname)
        upast=read.read_zonalmean_1lev(moddir+fname,datebegp,dateendp,plevuse)
        ufut=read.read_zonalmean_1lev(moddir+fname,datebegf,dateendf,plevuse)
        
        # check sizes
        if (upast.time.size != nmonthsp):
            print("somethings's wrong, "+str(nmonthspast)+
                  "expected, but got "+str(upast.time.size))
            sys.exit("failure at "+fname+" past period")
            
        if (ufut.time.size != nmonthsf):
            print("something's wrong, "+str(nmonthsfut)+
                 "expected, but got "+str(ufut.time.size))
            sys.exit("failure at "+fname+" future period")
            
        upastjja=cal.season_mean(upast,"ua",season="JJA")
        ufutjja=cal.season_mean(ufut,"ua",season="JJA")
        upastjja_ts=cal.season_ts(upast, "ua", season="JJA")
        ufutjja_ts=cal.season_ts(ufut,"ua", season="JJA")

        upastjja=upastjja.dropna("lat")
        ufutjja=ufutjja.dropna("lat")
        upastjjainterp=upastjja.interp(lat=np.linspace(-90,90,181),
                                      method='cubic')
        ufutjjainterp=ufutjja.interp(lat=np.linspace(-90,90,181),
                                    method='cubic')
        
        upastjja_ts = upastjja_ts.dropna("lat")
        ufutjja_ts = ufutjja_ts.dropna("lat")
        upastjja_tsinterp = upastjja_ts.interp(lat=np.linspace(-90,90,181),
                                               method='cubic')
        ufutjja_tsinterp = ufutjja_ts.interp(lat=np.linspace(-90,90,181),
                                            method='cubic')
        
        if (imem == 0):
            upast_jja_zm = xr.DataArray(np.zeros([nmems, upastjjainterp.lat.size]),
                                       coords=[np.arange(0,nmems,1),upastjjainterp["lat"]],
                                       dims=["member","lat"], name="upast_jja_zm")
            ufut_jja_zm = xr.DataArray(np.zeros([nmems, ufutjjainterp.lat.size]),
                                      coords=[np.arange(0,nmems,1), ufutjjainterp["lat"]],
                                      dims=["member","lat"], name="ufut_jja_zm")
            upast_jja_ts_zm = xr.DataArray(np.zeros([nmems, upastjja_tsinterp.time.size, upastjja_tsinterp.lat.size]),
                                          coords=[np.arange(0,nmems,1), upastjja_tsinterp["time"], upastjja_tsinterp["lat"]],
                                          dims=["member","time","lat"], name="upast_jja_ts_zm")
            ufut_jja_ts_zm = xr.DataArray(np.zeros([nmems, ufutjja_tsinterp.time.size, ufutjja_tsinterp.lat.size]),
                                          coords=[np.arange(0,nmems,1), ufutjja_tsinterp["time"], ufutjja_tsinterp["lat"]],
                                          dims=["member","time","lat"], name="ufut_jja_ts_zm")
            
            
        upast_jja_zm[imem,:]=upastjjainterp
        ufut_jja_zm[imem,:]=ufutjjainterp
        upast_jja_ts_zm[imem,:,:] = upastjja_tsinterp
        ufut_jja_ts_zm[imem,:,:] = ufutjja_tsinterp
    
    upast_jja_zm.to_netcdf(path="./DATA/uzm_"+modnames[imod]+".nc")
    ufut_jja_zm.to_netcdf(path="./DATA/uzm_"+modnames[imod]+".nc")
    upast_jja_ts_zm.to_netcdf(path="./DATA/uzm_"+modnames[imod]+".nc")
    ufut_jja_ts_zm.to_netcdf(path="./DATA/uzm_"+modnames[imod]+".nc")
            

CanESM2
ua_Amon_CanESM2_historical_rcp85_r10i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r11i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r12i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r13i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r14i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r15i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r16i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r17i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r18i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r19i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r1i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r20i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r21i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r22i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r23i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r24i1p1_195001-210012.nc
ua_Amon_CanESM2_historical_rcp85_r25i1p1_195001-210012.nc
ua_Amon